<a href="https://colab.research.google.com/github/bf319/Scaling_MPNNs/blob/main/attempt_4_before_meeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q git+https://github.com/deepmind/dm-haiku
%pip install -q jraph
%pip install -q git+https://github.com/deepmind/jaxline
%pip install -q ogb
%pip install -q dgl
%pip install -q optax
%pip install -q metis

     |████████████████████████████████| 75 kB 3.3 MB/s 
     |████████████████████████████████| 70 kB 5.5 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 78 kB 4.9 MB/s 
     |████████████████████████████████| 4.4 MB 13.8 MB/s 
     |████████████████████████████████| 136 kB 15.5 MB/s 


In [2]:
# # Set up the TPU
# import jax
# import jax.tools.colab_tpu
# jax.tools.colab_tpu.setup_tpu()

# device_count = jax.device_count() # 8 devices

# print(jax.devices())

KeyError: ignored

In [3]:
# from jax.experimental import maps
# from jax.experimental import PartitionSpec
# from jax.experimental.pjit import pjit

# import numpy as np

# mesh_shape = (4, 2)
# devices = np.asarray(jax.devices()).reshape(*mesh_shape)

# mesh = maps.Mesh(devices, ('x', 'y'))
# mesh

In [4]:
from ogb.nodeproppred import Evaluator
from ogb.nodeproppred import DglNodePropPredDataset


dataset = DglNodePropPredDataset(name = "ogbn-proteins")
split_idx = dataset.get_idx_split()
evaluator = Evaluator(name = 'ogbn-proteins')

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Downloaded 0.21 GB: 100%|██████████| 216/216 [00:51<00:00,  4.16it/s]


Extracting dataset/proteins.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:02<00:00,  2.73s/it]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00,  2.44it/s]


Saving...


In [5]:
# There is only one graph in Node Property Prediction datasets
ogbn_proteins_main_graph, ogbn_proteins_main_labels = dataset[0]

'''
  OGBN-Proteins
    #Nodes = 132,534
    #Edges = 39,561,252
    #Tasks = 112
    #Split Type = Species
    #Task Type = Binary classification
    #Metric = ROC-AUC

    Task:
      The task is to predict the presence of protein functions in a multi-label binary classification setup,
      where there are 112 kinds of labels to predict in total. 
      The performance is measured by the average of ROC-AUC scores across the 112 tasks.

    #Others:
      **undirected**
      **weighted**
      **typed (according to species)**

  (1) Nodes represent proteins
    (1.1) The proteins come from 8 species
      len(set(graph.ndata['species'].reshape(-1).tolist())) == 8
    (1.2) Each node has one feature associated with it (its species)
      graph.ndata['species'].shape == (#nodes, 1)
  
  (2) Edges indicate different types of biologically meaningful associations between proteins
    (2.1) All edges come with 8-dimensional features
      graph.edata['feat'].shape == (2 * #edges, 8)

'''
# Get split labels
train_label = dataset.labels[split_idx['train']]  # (86619, 112) -- binary values (presence of protein functions)
valid_label = dataset.labels[split_idx['valid']]  # (21236, 112) -- binary values (presence of protein functions)
test_label = dataset.labels[split_idx['test']]    # (24679, 112) -- binary values (presence of protein functions)

In [7]:
import dgl

'''
  Generate graph partition using metis, with balanced number of edges in each partition.
  Note: 
    The subgraphs do not contain the node/edge data in the input graph (https://docs.dgl.ai/generated/dgl.metis_partition.html)
'''
num_partitions = 100
dgl_graph_metis_partition = dgl.metis_partition(ogbn_proteins_main_graph, num_partitions, balance_edges = True)

Convert a graph into a bidirected graph: 6.261 seconds
Construct multi-constraint weights: 0.029 seconds
Metis partitioning: 35.167 seconds
Split the graph: 3.277 seconds
Construct subgraphs: 0.048 seconds


In [8]:
import numpy as np
import torch
import jraph
import jax.numpy as jnp

def dgl_graph_to_jraph(node_ids):
  # First add back the node and edge features
  dgl_graph_with_features = dgl.node_subgraph(ogbn_proteins_main_graph, node_ids)

  node_features = jnp.array(dgl_graph_with_features.ndata['species'])
  
  # I think it should be -- DGLGraph.edges uses the default (uv) format for edges
  # with u == source and v == destination
  # From https://docs.dgl.ai/generated/dgl.DGLGraph.edges.html#dgl.DGLGraph.edges
  senders = jnp.array(dgl_graph_with_features.edges()[0])
  receivers = jnp.array(dgl_graph_with_features.edges()[1])

  # Edges -- here we should include the 8-dimensional edge features
  edges = jnp.array(dgl_graph_with_features.edata['feat'])

  n_node = jnp.array([dgl_graph_with_features.num_nodes()])
  n_edge = jnp.array([dgl_graph_with_features.num_edges()])

  return jraph.GraphsTuple(
            nodes = node_features.astype(np.float32),
            senders = senders.astype(np.int32), 
            receivers = receivers.astype(np.int32),
            edges = edges.astype(np.float32),  
            n_node = n_node, 
            n_edge = n_edge,
            globals = None  # No global features
          )
  
def get_labels_for_subgraph(node_ids):
  return ogbn_proteins_main_labels.index_select(0, node_ids)

In [9]:
import haiku as hk
import jax
import jax.numpy as jnp
import jraph
import optax

In [10]:
from typing import Sequence

# See https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py for alternative to using GraphMapFeatures
# From https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py

mlp_sizes = (64, 128)
num_message_passing_steps = 3

@jraph.concatenated_args
def node_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = mlp_sizes), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

@jraph.concatenated_args
def edge_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Edge update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = mlp_sizes), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

def node_decoder_fn(feats: jnp.ndarray) -> jnp.ndarray:
  ## TODO -- Question
  ## Should I use softmax here then round the decoded values? (true labels are either 0 or 1) 
  net = hk.Sequential([hk.nets.MLP(output_sizes = [112]), jax.nn.relu])
  return net(feats)

def network_definition(graph):
  """Defines a graph neural network.
  Args:
    graph: Graphstuple the network processes.
  Returns:
    Decoded nodes.
  """

  embedder = jraph.GraphMapFeatures(
      embed_node_fn=hk.Linear(mlp_sizes[0]),
      embed_edge_fn=hk.Linear(mlp_sizes[1]),
      )
  graph = embedder(graph)

  gn = jraph.InteractionNetwork(
      update_node_fn=node_update_fn,
      update_edge_fn=edge_update_fn,
      include_sent_messages_in_node_update=True
      )

  for _ in range(num_message_passing_steps):
    graph = gn(graph)

  decoder = jraph.GraphMapFeatures(embed_node_fn = node_decoder_fn)
  
  processed_graph = decoder(graph)
  return processed_graph.nodes

In [15]:
import functools
import haiku as hk

from random import randint

# Try to follow this tutorial https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py
def compute_loss(params, graph, label, net):
  decoded_nodes = net.apply(params, graph) # Shape == label.shape
  
  ## TODO -- Question
  ## Should I round the predictions here?
  # predictions = jax.lax.round(decoded_nodes)
  predictions = decoded_nodes

  ##########################################################################################################################################################################
  # From https://colab.research.google.com/github/deepmind/educational/blob/master/colabs/summer_schools/intro_to_graph_nets_tutorial_with_jraph.ipynb#scrollTo=_Ld4b3D6Lwel
  ##########################################################################################################################################################################
  def compute_bce_with_logits_loss(x: jnp.ndarray, y: jnp.ndarray) -> jnp.ndarray:
    """Computes binary cross-entropy with logits loss.

    Combines sigmoid and BCE, and uses log-sum-exp trick for numerical stability.
    See https://stackoverflow.com/a/66909858 if you want to learn more.

    Args:
      x: Predictions (logits).
      y: Labels.

    Returns:
      Binary cross-entropy loss with mean aggregation.

    """
    max_val = jnp.clip(x, 0, None)
    loss = x - x * y + max_val + jnp.log(jnp.exp(-max_val) + jnp.exp((-x - max_val)))
    return loss.mean()
  ##########################################################################################################################################################################

  loss = compute_bce_with_logits_loss(predictions, label)
  return loss

def train(num_training_steps):
  # Transform the function (MPNN) into a pure function (with no side effects) so that it can be used with jax
  net = hk.without_apply_rng(hk.transform(network_definition))

  training_graph = dgl_graph_to_jraph(dgl_graph_metis_partition[0].ndata['_ID'])
  labels_training = get_labels_for_subgraph(dgl_graph_metis_partition[0].ndata['_ID'])
  labels_training = jnp.array(labels_training)

  params = net.init(jax.random.PRNGKey(42), training_graph)

  opt_init, opt_update = optax.adam(learning_rate = 1e-5)
  opt_state = opt_init(params)

  @jax.jit
  def update(params, opt_state, graph, targets):
    loss, grads = jax.value_and_grad(compute_loss)(params, graph, targets, net)
    updates, opt_state = opt_update(grads, opt_state)
    return optax.apply_updates(params, updates), opt_state, loss

  ## TODO - Question
  ## How to train only using the train_split?

  # Train
  for idx in range(num_training_steps):
    ## TODO - Question
    ## I should be training in each iteration over different graphs right?
    graph_idx = dgl_graph_to_jraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = get_labels_for_subgraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = jnp.array(labels_idx)

    params, opt_state, loss = update(params, opt_state, graph_idx, labels_idx)
    print('Loss training:', loss)

  return params

def evaluate(params, num_graphs_eval):
  net = hk.without_apply_rng(hk.transform(network_definition))
  # Evaluate
  accumulated_loss = 0.0
  accumulated_roc = 0

  for idx in range(num_graphs_eval):
    graph_idx = dgl_graph_to_jraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = get_labels_for_subgraph(dgl_graph_metis_partition[idx].ndata['_ID'])
    labels_idx = jnp.array(labels_idx)

    (loss, roc) = evaluate_graph(params, graph_idx, labels_idx, net)

    accumulated_loss += loss
    accumulated_roc += roc

  print(f'Average loss: {accumulated_loss / num_graphs_eval} | Average ROC: {accumulated_roc / num_graphs_eval}')

def evaluate_graph(params, graph, label, net):
  compute_loss_fn = jax.jit(functools.partial(compute_loss, net=net))

  decoded_nodes = net.apply(params, graph) # Shape == label.shape
  
  ## TODO -- Question
  ## Should I round the predictions here?
  # predictions = jax.lax.round(decoded_nodes)
  predictions = decoded_nodes

  loss = compute_loss_fn(params, graph, label)
  roc = evaluator.eval({"y_true": np.array(label), "y_pred": np.array(predictions)})['rocauc']

  print(f'Eval loss: {loss} | ROC: {roc}')
  return (loss, roc)

final_params = train(num_training_steps = 10)
evaluate(final_params, 5)

Loss training: 0.8661572
Loss training: 0.85732687
Loss training: 0.85012406
Loss training: 0.81998336
Loss training: 0.8535351
Loss training: 0.84924334
Loss training: 0.8748041
Loss training: 0.86169213
Loss training: 0.8686214
Loss training: 0.8390799
Eval loss: 0.844386100769043 | ROC: 0.4945177712628393
Eval loss: 0.8379455804824829 | ROC: 0.5063485438111911
Eval loss: 0.834211528301239 | ROC: 0.497473561267458
Eval loss: 0.8076703548431396 | ROC: 0.5121924628045824
Eval loss: 0.841364860534668 | ROC: 0.5018920366286708
Average loss: 0.8331156969070435 | Average ROC: 0.5024848751549482


In [ ]:
# ## TODO: Take into account that nodes might have been reshufled
# def predict_labels(params, net):
#   y_pred = None

#   for i in range(num_partitions):
#     graph = dgl_graph_to_jraph(dgl_graph_metis_partition[i].ndata['_ID'])

#     if i == 0:
#       y_pred = np.array(jax.nn.softmax(net.apply(params, graph)))
#     else:
#       if i % 5 == 0:
#         print(f'Reached iteration {i}')
#       y_pred = np.append(
#           y_pred,
#           np.array(jax.nn.softmax(net.apply(params, graph))),
#           axis = 0
#       )

#   return y_pred

In [ ]:
# (_, true_labels) = dataset[0]
# print(true_labels.shape)
# print(torch.tensor(predicted_labels).shape)

In [ ]:
# from ogb.nodeproppred import Evaluator

# evaluator = Evaluator(name = 'ogbn-proteins')
# final_results = evaluator.eval({
#     'y_true': true_labels,
#     'y_pred': torch.tensor(predicted_labels)
# })
# print(final_results)
# # print(evaluator.expected_input_format) 
# # print(evaluator.expected_output_format) 